# Tutorial 01: Build a Basic Chatbot

In this tutorial, you'll build your first LangGraph chatbot running entirely on local hardware with Ollama.

**What you'll learn:**
- `StateGraph`: The core LangGraph abstraction
- **Nodes**: Functions that process state
- **Edges**: Connections between nodes
- **State schema**: Defining what data flows through your graph
- **Reducers**: How state updates are handled (e.g., `add_messages`)

By the end, you'll understand the fundamental building blocks of LangGraph and have a working chatbot.

## Prerequisites

Make sure you have:
1. Ollama running (locally or on your LAN)
2. A model pulled (e.g., `ollama pull llama3.2:3b`)
3. The tutorial package installed (`pip install -e .`)

In [ ]:
# Verify Ollama connection
from langgraph_ollama_local import LocalAgentConfig

config = LocalAgentConfig()
print(f"Ollama server: {config.ollama.base_url}")
print(f"Model: {config.ollama.model}")

## Step 1: Define the State

Every LangGraph application starts with defining **State**. State is:
- A `TypedDict` that defines the schema of data flowing through your graph
- Shared between all nodes
- Updated by nodes as they process

For a chatbot, our state needs to track the conversation history:

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    """The state of our chatbot.
    
    Attributes:
        messages: The conversation history. Uses `add_messages` reducer
                  which appends new messages instead of overwriting.
    """
    messages: Annotated[list, add_messages]

### What is `add_messages`?

The `Annotated[list, add_messages]` syntax tells LangGraph how to update this field:

- **Without reducer**: New value overwrites old value
- **With `add_messages`**: New messages are appended to existing list

This is crucial for chatbots - we want to accumulate the conversation, not replace it!

## Step 2: Create the LLM

We'll use `ChatOllama` from LangChain to connect to our local Ollama server:

In [ ]:
from langchain_ollama import ChatOllama

# Create LLM using our config
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=config.ollama.temperature,
)

# Quick test
response = llm.invoke("Say 'hello' and nothing else.")
print(f"LLM test: {response.content}")

## Step 3: Define the Chatbot Node

A **node** is a function that:
1. Receives the current state as input
2. Does some processing
3. Returns updates to the state

Our chatbot node will:
1. Take the messages from state
2. Send them to the LLM
3. Return the LLM's response

In [ ]:
def chatbot(state: State) -> dict:
    """Process messages and generate a response.
    
    Args:
        state: Current state containing conversation messages
        
    Returns:
        Dictionary with new messages to add to state
    """
    # Invoke LLM with conversation history
    response = llm.invoke(state["messages"])
    
    # Return the response - add_messages will append it
    return {"messages": [response]}

## Step 4: Build the Graph

Now we assemble the pieces into a `StateGraph`:

1. Create a `StateGraph` with our State schema
2. Add the chatbot node
3. Connect edges (START → chatbot → END)
4. Compile the graph

In [ ]:
from langgraph.graph import StateGraph, START, END

# 1. Create the graph with our State schema
graph_builder = StateGraph(State)

# 2. Add the chatbot node
#    First arg: unique node name
#    Second arg: the function to call
graph_builder.add_node("chatbot", chatbot)

# 3. Add edges
#    START -> chatbot: Begin execution at the chatbot node
#    chatbot -> END: Finish after chatbot runs
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# 4. Compile the graph
graph = graph_builder.compile()

print("Graph compiled successfully!")

## Step 5: Visualize the Graph

LangGraph can render your graph as a diagram. This is incredibly helpful for understanding complex flows:

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not render graph: {e}")
    print("Install graphviz for visualization: brew install graphviz")
    # Fallback to ASCII
    print(graph.get_graph().draw_ascii())

## Step 6: Run the Chatbot

Now let's use our chatbot! We invoke the graph with an initial message:

In [ ]:
# Single invocation
result = graph.invoke({
    "messages": [("user", "What is LangGraph?")]
})

# Print the response
print("User: What is LangGraph?")
print(f"Assistant: {result['messages'][-1].content}")

## Step 7: Streaming Responses

For a better user experience, especially with slower local models, we can stream responses:

In [ ]:
def stream_response(user_input: str):
    """Stream the graph response for better UX."""
    print(f"User: {user_input}")
    print("Assistant: ", end="", flush=True)
    
    for event in graph.stream(
        {"messages": [("user", user_input)]},
        stream_mode="values"
    ):
        # Get the last message
        last_message = event["messages"][-1]
        # Only print AI messages (skip user message echo)
        if hasattr(last_message, 'content') and last_message.type == "ai":
            print(last_message.content)

# Test streaming
stream_response("Explain neural networks in 2 sentences.")

## Step 8: Interactive Chat Loop

Let's create an interactive chat session. Note: this won't maintain memory between messages yet (we'll add that in Tutorial 03).

In [ ]:
def chat():
    """Simple interactive chat loop."""
    print("Chat with your local LLM! Type 'quit' to exit.")
    print("-" * 50)
    
    while True:
        try:
            user_input = input("You: ")
            if user_input.lower() in ["quit", "exit", "q"]:
                print("Goodbye!")
                break
            
            # Invoke graph
            result = graph.invoke({"messages": [("user", user_input)]})
            print(f"Assistant: {result['messages'][-1].content}")
            print()
            
        except KeyboardInterrupt:
            print("\nGoodbye!")
            break

# Uncomment to run interactive chat:
# chat()

## Complete Code

Here's the complete chatbot in one cell:

In [ ]:
# Complete basic chatbot implementation

from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama
from langgraph_ollama_local import LocalAgentConfig

# 1. State definition
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 2. LLM setup
config = LocalAgentConfig()
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
)

# 3. Node definition
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# 4. Graph construction
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

# 5. Use it!
result = graph.invoke({"messages": [("user", "Hello!")]})
print(result["messages"][-1].content)

## Key Concepts Recap

| Concept | Description |
|---------|-------------|
| **StateGraph** | The container for your graph definition |
| **State** | TypedDict defining data that flows through the graph |
| **Node** | A function that processes state and returns updates |
| **Edge** | Connections between nodes (including START and END) |
| **Reducer** | Function that determines how state updates are merged (e.g., `add_messages`) |
| **compile()** | Converts the graph definition into an executable graph |

## What's Next?

In [Tutorial 02: Tool Calling](02_tool_calling.ipynb), you'll learn:
- How to define tools for your agent
- The ReAct (Reasoning + Acting) pattern
- Conditional edges for routing
- Building agents that can take actions